# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [12]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [13]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [14]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [15]:
step1 = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity":"sum"})
step1 = pd.DataFrame(step1)
step1

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [16]:
table = pd.pivot_table(data, values='Quantity', index=['ProductName'],columns=['CustomerID'], fill_value=0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [17]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), index=table.columns, columns=table.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [18]:
similar_200 = distances[200].sort_values(ascending=False)[1:6]
similar_200

CustomerID
1920    0.094414
1072    0.092610
3253    0.090094
3317    0.090094
3909    0.089695
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [19]:
step5 = step1.loc[[i for i in similar_200.index]]
#step5.1 = step1.loc[[1920, 1072, 3253, 3317, 3909]]
step5

Quantity
CustomerID ProductName                                
1072       Appetizer - Mini Egg Roll, Shrimp         1
           Appetizer - Sausage Rolls                 1
           Beans - Kidney, Canned                    1
           Beef - Montreal Smoked Brisket            1
           Beef - Striploin Aa                       1
...                                                ...
3909       Wine - Charddonnay Errazuriz              2
           Wine - Fume Blanc Fetzer                  1
           Wine - Magnotta, Merlot Sr Vqa            1
           Wine - Toasted Head                       1
           Wine - Valpolicella Masi                  1

[261 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [20]:
step6 = step5.groupby('ProductName').agg({"Quantity":"sum"}).sort_values('Quantity', ascending=False)
step6.head()

,Quantity
ProductName,
Soup - Campbells Bean Medley,3
Lamb - Ground,3
Pomello,3
Bay Leaf,3
Pork - Kidney,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [22]:
step7 = step6.merge(step1.loc[200], how = 'left', on = 'ProductName')
step7_filtered = step7[step7['Quantity_y'].isnull()].head()
step7_filtered

,Quantity_x,Quantity_y
ProductName,,
Soup - Campbells Bean Medley,3,NaN
Pomello,3,NaN
Bay Leaf,3,NaN
Pork - Kidney,3,NaN
Muffin - Carrot Individual Wrap,3,NaN


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [32]:
dic={}
customers_id= list(set(data['CustomerID'].tolist()))
for c in customers_id:
    similar = distances[c].sort_values(ascending=False)[1:6]
    similar_products = step1.loc[[i for i in similar.index]]
    agg = similar_products.groupby('ProductName').agg({"Quantity":"sum"}).sort_values('Quantity', ascending=False)
    products = agg.merge(step1.loc[c], how = 'left', on = 'ProductName')
    products_recomendations = products[products['Quantity_y'].isnull()].head()
    dic[c]=list(products_recomendations['Quantity_x'].keys())

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [34]:
df = pd.DataFrame(dic)
df.T.head()

,0,1,2,3,4
83973,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Cocoa Butter,"Pepper - Paprika, Hungarian"
59399,Lettuce - California Mix,"Nut - Pistachio, Shelled",Nantucket - Pomegranate Pear,"Shrimp - Baby, Warm Water",Bagel - Plain
92168,Pecan Raisin - Tarts,Bandage - Flexible Neon,Beef Wellington,Sauerkraut,Bread - Calabrese Baguette
49159,"Doilies - 5, Paper","Wine - Magnotta, Merlot Sr Vqa","Pasta - Penne, Rigate, Dry",Ezy Change Mophandle,Cookies - Assorted
18441,"Mushroom - Trumpet, Dry",Nantuket Peach Orange,Veal - Inside,Initation Crab Meat,Island Oasis - Mango Daiquiri


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [42]:
distances_1 = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cosine'))), index=table.columns, columns=table.columns)
dic_1={}
customers_id= list(set(data['CustomerID'].tolist()))
for c1 in customers_id:
    similar = distances_1[c1].sort_values(ascending=False)[1:6]
    similar_products = step1.loc[[i for i in similar.index]]
    agg = similar_products.groupby('ProductName').agg({"Quantity":"sum"}).sort_values('Quantity', ascending=False)
    products = agg.merge(step1.loc[c1], how = 'left', on = 'ProductName')
    products_recomendations = products[products['Quantity_y'].isnull()].head()
    dic_1[c1]=list(products_recomendations['Quantity_x'].keys())
df1 = pd.DataFrame(dic_1)
df1.T.head()

,0,1,2,3,4
83973,"Beef - Ground, Extra Lean, Fresh","Cheese - Brie,danish","Mushroom - Trumpet, Dry","Seedlings - Mix, Organic","Wine - White, Mosel Gold"
59399,Ezy Change Mophandle,Anchovy Paste - 56 G Tube,Garlic - Peeled,Water - Aquafina Vitamin,Cake - Mini Cheesecake
92168,"Beer - Alexander Kieths, Pale Ale",Nantucket - Pomegranate Pear,"Crab - Dungeness, Whole",Foam Cup 6 Oz,Cake - Mini Cheesecake
49159,"Turnip - White, Organic",Cookies - Assorted,Pernod,Chips Potato All Dressed - 43g,Rice - Long Grain
18441,Ecolab - Lime - A - Way 4/4 L,"Beans - Kidney, Red Dry",Tea - Jasmin Green,Oil - Safflower,"Cheese - Boursin, Garlic / Herbs"
